# Particle Swarm Optimization

Sources:
- http://www.swarmintelligence.org/tutorials.php
- https://github.com/tisimst/pyswarm/blob/master/pyswarm/pso.py

At iteration $k$, particle $i$ has posistion $x^i_k$ defined recursively as:

$$x^i_{k+1} = x^i_k + v^i_{k+1}$$

where $v$ is its velocity defined as follows:

$$ v^i_{k+1} = w_k \cdot v^i_{k} + c_1 r_1 (p^i_k - x^i_k) + c_2 r_2 (p^g_k - x^i_k)$$

Where:

- $p^i_k$ is the particle's best known position
- $p^g_k$ is the global (all particles) best known position
- $w_k$ is an inertia parameter
- $c_1$ is the cognitive parameter
- $c_2$ is the social parameter
- $r_1, r_2$ are random numbers in $(0, 1)$

In plain English, the velocity update is defined as:

new_velocity = (inertia * previous_velocity) + (move in the direction of the personal best) + (move in the direction of the global best)

In [1]:
# import two useful libraries/modules
import numpy as np
import random

In [2]:
# -- Particle Class

class Particle:
    """A Python Class for a simple particle."""
    
    def __init__(self, upper, lower, ndim, c1 = 1, c2 = 2, w = .5):
        '''Initiate a Particle with an upper and lower bound, and a number of dimensions.'''
        # Initiate position and velocity randomly
        self.position = np.array([random.uniform(lower, upper) for _ in range(ndim)])
        self.velocity = np.array([random.uniform(-1, 1) for _ in range(ndim)])
        # These attributes are here to store the "memory" of the function
        self.personal_best_position = self.position # initial position
        self.personal_best_error = np.inf # infinity
        self.c1 = c1
        self.c2 = c2
        self.w = w
    
    def __str__(self):
        '''what will be called when you print a Particle instance, could be useful for debugging.'''
        return("I am a particle with best position {},\ncurrent position {},\nand velocity {}.\n"
               .format(self.personal_best_position, self.position, self.velocity))       
    
    def current_error(self, function):
        '''Compute the current "fitness" value of a particle, given a function. Updates it if needed.'''
        current_fitness = function(self.position)
        if current_fitness < self.personal_best_error:
            self.personal_best_error = current_fitness
            self.personal_best_position = self.position
        return current_fitness
    
    def update_velocity(self, global_best):
        '''update a particle's velocity'''
        # two random coefficients in uniform([0, 1])
        r1 = random.random()
        r2 = random.random()
        self.velocity = (self.w*self.velocity + # inertia
                         self.c1*r1*(self.personal_best_position - self.position) + # cognitive term
                         self.c2*r2*(global_best - self.position)) # social term
            
    def move(self):
        '''Moves a Particle at a new iteration. Deals with out-of-bound cases (are they ok or not?)'''
        self.position = self.position + self.velocity
        # need to deal with when the particle goes out of bound...

In [3]:
# --- Solver Class
class PSO:    
    """Solver instance for Particle Swarm Optimizer. (Basically the mastermind which commands the Particles.)
    Initiated with:
    - num_particles: how many particles to use?
    - function: function to optimize.
    - n_iter: number of iterations to be performed.
    - lower, upper: lower and upper bounds for the search space (for now, a box i.e. they are the same accross all dimensions.
    - ndim: dimensionality of the search space.)"""
    
    def __init__(self, num_particles, function, n_iter, lower = -10, upper = 10, ndim = 3,
                 c1 = 1, c2 = 2, w = .5):
        '''Initiate the solver'''
        # create all the Particles, stored in a list.
        self.particles = [Particle(lower, upper, ndim, c1, c2, w) for _ in range(num_particles)]
        # store global best and associated error
        self.global_best = np.array([])
        self.global_best_error = np.inf # infinity
        self.function = function # function to be optimised
        self.n_iter = n_iter # num of iterations
        
    def update_best(self):
        '''Find the new best global position and update it.'''
        for particle in self.particles:
            # ask all particles for their current error. If better than current global best, update.
            if particle.current_error(self.function) < self.global_best_error:
                self.global_best = particle.position
                self.global_best_error = particle.current_error(self.function)
                
    def move_particles(self):
        '''Run one iteration of the algorithm. Update particles velocity and move them.'''
        for particle in self.particles:
            particle.update_velocity(self.global_best)
            particle.move()
    
    def __str__(self):
        '''Print best global position when calling `print(pso instance)`'''
        return """Current best position: {}
        With error: {}""".format(self.global_best, self.global_best_error)
    
    def go(self):
        '''Run the algorithm and print the result. By default, update us every 50 iterations.'''
        print("Running the PSO algorithm with {} particles, for {} iterations.\n".
              format(len(self.particles), self.n_iter))
        
        for iteration in range(self.n_iter):
            pso.update_best()
            pso.move_particles()
            if iteration % 50 == 0:
                print("Iteration number " + str(iteration))
                print("Current best error: " + str(self.global_best_error))
                print("\n")
                
        print("Found minimum at {} with value {}.".format(self.global_best, self.global_best_error))

### Tests

In [4]:
def sumsquares(x):
    return(sum([i**2 for i in x]))

In [5]:
pso = PSO(100, sumsquares, n_iter = 100)
pso.go()

Running the PSO algorithm with 100 particles, for 100 iterations.

Iteration number 0
Current best error: 2.9735249206556267


Iteration number 50
Current best error: 2.0465473224924796e-12


Found minimum at [3.08408503e-12 4.99070051e-12 1.35451975e-12] with value 3.625339583073418e-23.


In [6]:
for p in pso.particles[0:10]:
    print(p)

I am a particle with best position [ 1.10499094e-11  2.53705355e-12 -1.63068192e-11],
current position [ 6.75208651e-12 -4.06786197e-11  4.09852188e-11],
and velocity [-4.29782294e-12 -4.32156732e-11  5.72920380e-11].

I am a particle with best position [ 2.07753516e-12 -2.06457687e-11  8.90392423e-12],
current position [-2.20800553e-11 -9.65459435e-12  1.78378522e-11],
and velocity [ 5.26278371e-11 -1.92513940e-12 -2.55190936e-11].

I am a particle with best position [4.85981021e-11 2.39873430e-11 1.20221759e-12],
current position [4.59986583e-11 2.66758061e-11 8.68203185e-13],
and velocity [-3.97447242e-11 -4.64711255e-11  4.11468841e-13].

I am a particle with best position [ 3.87987327e-13  3.90697530e-12 -1.17774345e-11],
current position [ 1.84436151e-13  2.49563349e-11 -1.21286600e-11],
and velocity [-2.03551176e-13  2.10493596e-11 -3.51225484e-13].

I am a particle with best position [7.44331368e-12 2.93374429e-11 1.20276301e-10],
current position [2.68032391e-11 8.66756914e-11

In [7]:
def shiftedsumsquares(x):
    return(sum([(i+2)**2 for i in x])) # minimum should be all -2

In [8]:
pso = PSO(20, shiftedsumsquares, n_iter = 100) # works well even with 20 swarms!
pso.go()

Running the PSO algorithm with 20 particles, for 100 iterations.

Iteration number 0
Current best error: 10.952760883585142


Iteration number 50
Current best error: 2.783606501881347e-10


Found minimum at [-2. -2. -2.] with value 1.1795060584622466e-19.


Trying the function $f(x) := f(x_1, x_2) = (x_1 + 2 x_2 − 3)^2 + (x_1 − 2)^2$

In [9]:
def newfunct(x):
    return (x[0] + 2*x[1] - 3)**2 + (x[0] - 2)**2

In [10]:
pso = PSO(10, newfunct, 100, ndim=2)
pso.go()

Running the PSO algorithm with 10 particles, for 100 iterations.

Iteration number 0
Current best error: 0.16236549651619692


Iteration number 50
Current best error: 7.386246396788135e-12


Found minimum at [2.  0.5] with value 2.4573205784694672e-23.


## Parallelizing

In [2]:
import multiprocessing

In [3]:
multiprocessing.cpu_count()

4

The `Particle` class doesn't change, it only looses the capacity to evaluate its own fitness.

In [4]:
# -- Particle Class

class Particle:
    """A Python Class for a simple particle."""
    
    def __init__(self, upper, lower, ndim, c1 = 1, c2 = 2, w = .5):
        '''Initiate a Particle with an upper and lower bound, and a number of dimensions.'''
        # Initiate position and velocity randomly
        self.position = np.array([random.uniform(lower, upper) for _ in range(ndim)])
        self.velocity = np.array([random.uniform(-1, 1) for _ in range(ndim)])
        # These attributes are here to store the "memory" of the function
        self.personal_best_position = self.position # initial position
        self.personal_best_error = np.inf # infinity
        self.c1 = c1
        self.c2 = c2
        self.w = w
    
    
    def __str__(self):
        '''what will be called when you print a Particle instance, could be useful for debugging.'''
        return("I am a particle with best position {},\ncurrent position {},\nand velocity {}.\n"
               .format(self.personal_best_position, self.position, self.velocity))
    
    
    def update_velocity(self, global_best):
        '''update a particle's velocity'''
        # two random coefficients in uniform([0, 1])
        r1 = random.random()
        r2 = random.random()
        self.velocity = (self.w*self.velocity + # inertia
                         self.c1*r1*(self.personal_best_position - self.position) + # cognitive term
                         self.c2*r2*(global_best - self.position)) # social term
        
            
    def move(self):
        '''Moves a Particle at a new iteration. Deals with out-of-bound cases (are they ok or not?)'''
        self.position = self.position + self.velocity
        # need to deal with when the particle goes out of bound...

In [5]:
# --- Solver Class
class PSO:
    """
    +----------------------------------------------------------+
    + Parallel Solver instance for Particle Swarm Optimizer.   +
    + (Basically the mastermind which commands the Particles.) +
    +----------------------------------------------------------+
    
    Instantiated with:
    -----------------
    - num_particles: how many particles to use?
    
    - function: function to minimize. IT MUST BE of the form:
                function np.array[coordinates] --- f() ---> [list] (or np.array)
                typically, the function returns a list comprehension.
                
    - n_iter: number of iterations to be performed. To be replaced/completed with convergence criterions.
    
    - ndim: dimensionality of the search space.
    
    - lower, upper: lower and upper bounds for the search space
    (for now, it's a box i.e. they are the same accross all dimensions.)
    
    - c1, c2, w: cognitive, social, and inertia parameters. To be tuned to the specific problem.
    
    - parallel: whether to evaluate the fitness of particles in parallel. `False` by default as a speed-boost is
    unlikely in most simple settings.
    
    """
    
    def __init__(self, num_particles, function, n_iter, ndim, lower = -10, upper = 10,
                 c1 = 1, c2 = 2, w = .5, parallel = False):
        '''Initiate the solver'''
        # create all the Particles, stored in a list.
        self.particles = [Particle(lower, upper, ndim, c1, c2, w) for _ in range(num_particles)]
        self.fitnesses = np.array([])
        # store global best and associated error
        self.global_best = np.array([])
        self.global_best_error = np.inf # infinity
        self.function = function # function to be optimised
        self.n_iter = n_iter # num of iterations
        self.parallel = parallel
        if parallel:
            self.pooler = Pool(multiprocessing.cpu_count() - 1)
        
    def get_fitnesses(self):
        # Evaluate all fitnesses in parallel, or not.
        if self.parallel:
            fitnesses = self.pooler.apply(self.function, [[part.position for part in self.particles]])
            self.fitnesses = np.array(fitnesses)
            
        else :
            fitnesses = self.function([part.position for part in self.particles])
            self.fitnesses = np.array(fitnesses)
    
    def update_particles(self):
        '''update particle best known personal position'''
        for i in range(len(self.fitnesses)):
            if self.fitnesses[i] < self.particles[i].personal_best_error:
                self.particles[i].personal_best_error = self.fitnesses[i]
                self.particles[i].personal_best_position = self.particles[i].position          
        
    def update_best(self):
        '''Find the new best global position and update it.'''
        if np.any(self.fitnesses < self.global_best_error):
            self.global_best_error = np.min(self.fitnesses)
            self.global_best = self.particles[np.argmin(self.fitnesses)].position
                      
    def move_particles(self):
        '''Run one iteration of the algorithm. Update particles velocity and move them.'''
        for particle in self.particles:
            particle.update_velocity(self.global_best)
            particle.move()
    
    def __str__(self):
        '''Print best global position when calling `print(pso instance)`'''
        return """Current best position: {}
        With error: {}""".format(self.global_best, self.global_best_error)
    
    def run(self, verbose = True):
        '''Run the algorithm and print the result. By default, update us every 50 iterations.'''
        
        if verbose:
            print("Running the PSO algorithm in parallel with {} particles, for {} iterations.\n".
                  format(len(self.particles), self.n_iter))
        
        for iteration in range(self.n_iter):
            # this happens in parallel (synchronous only rn)
            self.get_fitnesses()
            # this doesn't
            self.update_particles()
            self.update_best()
            self.move_particles()
            
            if (iteration % 50 == 0) & verbose==True:
                print("Iteration number " + str(iteration))
                print("Current best error: " + str(self.global_best_error))
                print("\n")
                
        if verbose:
            print("Found minimum at {} with value {}.".format(self.global_best, self.global_best_error))
            
        return(self.global_best)

## Tests and benchmarks

In [6]:
# This function is cheap to evaluate
def quad_function(li):
    return [(x[0] + 2*x[1] - 3)**2 + (x[0] - 2)**2 for x in li]
from multiprocessing import Pool

In [7]:
pso = PSO(20, quad_function, 100, 2)
pso.run()

Running the PSO algorithm in parallel with 20 particles, for 100 iterations.

Iteration number 0
Current best error: 6.710500951250321


Iteration number 50
Current best error: 1.259071556239094e-13


Found minimum at [2.  0.5] with value 1.0853776030652965e-22.


array([2. , 0.5])

The [Rosenbrock Function](https://en.wikipedia.org/wiki/Rosenbrock_function)

$f(x,y)=(1-x)^{2}+100(y-x^{2})^{2}$

In [8]:
def rosenbrock(li):
    def f(x):
        return (1-x[0])**2 + 100*(x[1] - x[0]**2)**2
    return [f(x) for x in li]

In [9]:
pso = PSO(20, rosenbrock, 100, ndim = 2)
pso.run()

Running the PSO algorithm in parallel with 20 particles, for 100 iterations.

Iteration number 0
Current best error: 793.1763569363916


Iteration number 50
Current best error: 0.2364910306654901


Found minimum at [0.99999995 0.9999999 ] with value 3.597231035066068e-15.


array([0.99999995, 0.9999999 ])

High dimensional Rosenbrock

${\displaystyle f(\mathbf {x} )=\sum _{i=1}^{N-1}[100(x_{i+1}-x_{i}^{2})^{2}+(1-x_{i})^{2}]\quad {\mbox{where}}\quad \mathbf {x} =[x_{1},\ldots ,x_{N}]\in \mathbb {R} ^{N}.}$

In [10]:
def high_dim_rosenbrock(li):
    def f(x):
        return sum([(1-x[i])**2 + 100*(x[i+1] - x[i]**2)**2 for i in range(len(x) - 1)])
    return [f(x) for x in li]

In [11]:
high_dim_rosenbrock([np.array([0, 0, 0]), np.array([1, 1, 1])])

[2, 0]

In [32]:
pso = PSO(100, high_dim_rosenbrock, 100, ndim = 5)
sol = pso.run()

Running the PSO algorithm in parallel with 100 particles, for 100 iterations.

Iteration number 0
Current best error: 23500.485110354603


Iteration number 50
Current best error: 2.6552350773821664e-05


Found minimum at [1.         1.00000001 1.00000001 1.00000002 1.00000005] with value 2.1480739759907733e-15.


## Griewank function

$$f(x) = 1+{\frac  {1}{4000}}\sum _{{i=1}}^{n}x_{i}^{2}-\prod _{{i=1}}^{n}\cos \left({\frac  {x_{i}}{{\sqrt  {i}}}}\right)$$

In [14]:
def griewank(li):
    def f(x):
        return 1 + (1/4000) * sum([xi**2 for xi in x]) - np.product([np.cos(x[i]/np.sqrt(i+1)) for i in range(len(x))])
    return [f(x) for x in li]

In [26]:
pso = PSO(500, griewank, 100, ndim = 3, parallel=False, c1 = 3, c2 = .5)
pso.run()

Running the PSO algorithm in parallel with 500 particles, for 100 iterations.

Iteration number 0
Current best error: 0.0793802308709628


Iteration number 50
Current best error: 0.007396040384200164


Found minimum at [-0.00229361 -0.00068673 -0.00784785] with value 1.3029783071272227e-05.


array([-0.00229361, -0.00068673, -0.00784785])

For this function, hyperparameter tuning matters a lot!
(here, high c1 to let particles explore more!)

## Schaffer's F6

$$f(x)=0.5+\frac{sin^2(\sqrt{x_1^2 + x_2^2})-0.5}{[1+0.001 \cdot (x_1^2 + x_2^2)]^2}$$

In [33]:
def schaffer_f6(li):
    def f(x):
        return .5 + ((np.sin(np.sqrt(x[0]**2 + x[1]**2))**2) - .5)/((1 + 0.001*(x[0]**2 + x[1]**2))**2)
    return [f(x) for x in li]

In [36]:
schaffer_f6([[0, 0], [1, 0], [1, 1]])

[0.0, 0.7076578948260244, 0.9737845308015942]

In [38]:
pso = PSO(100, schaffer_f6, 100, 2)
pso.run()

Running the PSO algorithm in parallel with 100 particles, for 100 iterations.

Iteration number 0
Current best error: 0.010156177211700756


Iteration number 50
Current best error: 8.05111532997671e-12


Found minimum at [3.86153945e-09 2.74571391e-09] with value 0.0.


array([3.86153945e-09, 2.74571391e-09])

# Solver is in the `PSPSO.py` module !

In [27]:
from PSPSO import Particle, PSO

In [28]:
pso = PSO(100, rosenbrock, 100, ndim = 2)

In [29]:
pso.run()

Running the PSO algorithm in parallel with 100 particles, for 100 iterations.

Iteration number 0
Current best error: 8.40742499797306


Iteration number 50
Current best error: 4.005769522933947e-12


Found minimum at [1. 1.] with value 9.393428282096142e-25.


array([1., 1.])